In [1]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.utils import get_column_letter
from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyxl.styles import PatternFill

In [2]:
# Read in the data 
complete_cases = pd.read_excel('data/complete_cases.xlsx')

In [3]:
complete_cases['post_pct'] = complete_cases['post'].rank(pct=True)
complete_cases['post_pct'].describe()

count    80.000000
mean      0.506250
std       0.290375
min       0.012500
25%       0.264062
50%       0.506250
75%       0.756250
max       1.000000
Name: post_pct, dtype: float64

In [4]:
complete_cases.head()

,email,cohort,pre,post,age,sex,education,satisfaction,study_hours,post_pct
0,smehaffey0@creativecommons.org,4,485,494,32,Male,Bachelor's,2,36.6,0.67500
1,kbridgnell5@webnode.com,1,484,503,32,Female,Bachelor's,6,32.2,0.88125
2,abotham6@usda.gov,1,497,501,30,Female,Bachelor's,10,32.3,0.82500
3,szeal7@pinterest.com,2,451,457,32,Male,Bachelor's,4,26.2,0.12500
4,ikidby8@who.int,4,452,448,29,Female,Bachelor's,9,27.9,0.08750


In [5]:
# Create a new workbook and select the worksheet
wb = Workbook()

# Assign the active worksheet to ws
ws = wb.active

In [6]:
ws['A1'] = "Average pre score"
ws['B1'] = round(complete_cases['pre'].mean(), 2)  # Round output to two decimals

ws['A2'] = "Average post score"
ws['B2'] = round(complete_cases['post'].mean(), 2)

In [7]:
ws.column_dimensions['A'].width = 16

In [8]:
ws2 = wb.create_sheet(title='data')

In [9]:
for row in dataframe_to_rows(complete_cases, index=False, header=True):
    ws2.append(row)

In [10]:
post_pct_loc = complete_cases.columns.get_loc('post_pct') + 1
post_pct_col = get_column_letter(post_pct_loc) 
post_pct_col

'J'

In [11]:
number_format = '0.0%'

for cell in ws2[post_pct_col]:
    cell.number_format = number_format

In [12]:
# Specify desired table formatting
style = TableStyleInfo(name='TableStyleMedium9', showRowStripes=True)

# Name and identify range of table
table = Table(displayName='contestants', 
              ref='A1:' + get_column_letter(ws2.max_column) + str(ws2.max_row))
              
# Apply styling and insert in worksheet 
table.tableStyleInfo = style
ws2.add_table(table)

In [13]:
# Define conditional formatting style
green_fill = PatternFill(start_color="B9E8A2", end_color="B9E8A2", fill_type="solid")
yellow_fill = PatternFill(start_color="FFF9D4", end_color="FFF9D4", fill_type="solid")

# Loop through data table and conditonally apply formatting
for row in ws2.iter_rows(min_row=2, min_col=1, max_col=len(complete_cases.columns)):
    post_pct = row[post_pct_loc - 1].value # Convert index to 0-based indexing
    if post_pct > .9:
        for cell in row:
            cell.fill = green_fill
    elif post_pct > .7:
        for cell in row:
            cell.fill = yellow_fill

In [14]:
for column in ws2.columns:
    max_length = 0
    column_letter = column[0].column_letter
    for cell in column:
        try:
            if len(str(cell.value)) > max_length:
                max_length = len(cell.value)
        except:
            pass
    adjusted_width = (max_length + 2) * 1.2
    ws2.column_dimensions[column_letter].width = adjusted_width

In [15]:
wb.save('output/m_03_v_07_output.xlsx')